# Notebook for using an LSTM to diagnose ASD 

In [28]:
%load_ext autoreload
%autoreload 2

import os
import json
import pickle
import random
random.seed(42)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, AveragePooling2D, AveragePooling1D
from keras.metrics import BinaryAccuracy
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import sys
sys.path.append('../src/features')

from subject import Subject

cur_dir = os.getcwd()
abide_dir = os.path.dirname(os.path.dirname(cur_dir)) + '/abide/'
subjects_dir = os.path.dirname(cur_dir) + '/data/ABIDEI_subjects/'
trs_save_file = save_dir = os.path.dirname(cur_dir) + '/data/dicts/ABIDEI_site_trs.json'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Dictionary with TRs for each scanning site
with open(trs_save_file) as json_file:
    site_trs = json.load(json_file)

In [3]:
def open_pickle(f):
    file = open(f,'rb')
    o = pickle.load(file)
    file.close()
    return o

def load_subjects(subject_folder):
    subjects = list()
    for f in os.listdir(subject_folder):
        subjects.append(open_pickle(os.path.join(subject_folder, f)))
    return subjects

In [4]:
subjects = load_subjects(subjects_dir)

In [5]:
# For now let's just look at sites with Trs of 2s
clean_subjects = list()
asd_c = 0
for s in subjects:
    if(site_trs[s._site_id] == 2):
        clean_subjects.append(s)
        # Note dx group 1 is positive for ASD
        if(s._label_dict['dx_group'] == 1):
            asd_c += 1

In [6]:
print(f'{asd_c} subjects with ASD out of {len(clean_subjects)} subjects in clean list')

253 subjects with ASD out of 548 subjects in clean list


# Randomly extract sections of even length from scan to use for features
* Doing 3 mins trying to replicate https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5669262/

In [13]:
# Since scans are 2s apart 90 scans is 3 mins
L = 90
# Number of clips per subject
N=10
# Number of ROIs
N_rois = 200
feat_name = 'filt_noglobal_roi_200_Cradd'
def extract_feat_sections(s, feat_name=feat_name, L=L, N=N):
    data = s._data_dict[feat_name]
    feat_secs = list()
    for i in range(N):
        r = int(random.random() * (len(data) - L))
        feat_secs.append(data[r:r+L])
    return np.array(feat_secs)

def create_dataset(subjects, feat_name=feat_name, L=L,N=N):
    X = list()
    Y = list()
    for s in subjects:
        feat_secs = extract_feat_sections(s)
        X.extend(feat_secs)
        # 1 is still classified ASD and 0 is control
        if(s._label_dict['dx_group'] == 1):
        # if(s._sex == 1):
            Y.extend([1]*len(feat_secs))
        else:
            Y.extend([0]*len(feat_secs))
    assert len(X) == len(Y)
    X_ar = np.array(X).reshape(len(X), L, N_rois)
    Y_ar = np.array(Y)
    return X_ar, Y_ar

In [14]:
# In original work 10 fold cross val used with proportion of subjects from each site was approximately the same in all folds
# To start will just randomly split subjects into groups
val_per = .05
test_per = .1
train_subs, val_subs = train_test_split(clean_subjects, test_size=val_per + test_per, random_state=42)
val_subs, test_subs = train_test_split(val_subs, test_size=test_per/(val_per + test_per), random_state=43)
print(f'{len(train_subs)} subjects for training.')
print(f'{len(val_subs)} subjects for validation')
print(f'{len(test_subs)} subjects for testing')

465 subjects for training.
27 subjects for validation
56 subjects for testing


In [15]:
train_X, train_Y = create_dataset(train_subs)
val_X, val_Y = create_dataset(val_subs)
test_X, test_Y = create_dataset(test_subs)
print(f'{len(train_X)} training examples')
print(f'{len(val_X)} validation examples')
print(f'{len(test_X)} testing examples')

4650 training examples
270 validation examples
560 testing examples


# Create LSTM model

In [40]:
# create and fit the LSTM network
# hidden_nodes = int(2/3 * (N_rois * L))
hidden_nodes = 16
model = Sequential()
model.add(LSTM(hidden_nodes, input_shape=(L, N_rois), return_sequences=True, dropout=0.5))
model.add(Dropout(0.5))
# model.add(LSTM(hidden_nodes))
# model.add(Dropout(0.25))
# model.add(AveragePooling2D(pool_size=(L, 1)))
model.add(AveragePooling1D(pool_size=L))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 90, 16)            13888     
_________________________________________________________________
dropout_13 (Dropout)         (None, 90, 16)            0         
_________________________________________________________________
average_pooling1d_2 (Average (None, 1, 16)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 1, 1)              17        
Total params: 13,905
Trainable params: 13,905
Non-trainable params: 0
_________________________________________________________________


In [45]:
batch_size = 32
epochs = 5
model.fit(train_X, train_Y, batch_size=batch_size, epochs=epochs, validation_data=(val_X, val_Y))

Epoch 1/5
146/146 [==============================] - 23s 160ms/step - loss: 0.5457 - accuracy: 0.7658 - val_loss: 0.6035 - val_accuracy: 0.6407
Epoch 2/5
146/146 [==============================] - 24s 163ms/step - loss: 0.5286 - accuracy: 0.7740 - val_loss: 0.5988 - val_accuracy: 0.6481
Epoch 3/5
146/146 [==============================] - 24s 164ms/step - loss: 0.5129 - accuracy: 0.7843 - val_loss: 0.5896 - val_accuracy: 0.6630
Epoch 4/5
146/146 [==============================] - 25s 171ms/step - loss: 0.4996 - accuracy: 0.7920 - val_loss: 0.6184 - val_accuracy: 0.6148
Epoch 5/5
146/146 [==============================] - 25s 171ms/step - loss: 0.4813 - accuracy: 0.8049 - val_loss: 0.5427 - val_accuracy: 0.6926


In [46]:
# evaluate the model
_, train_acc = model.evaluate(train_X, train_Y, verbose=0)
_, test_acc = model.evaluate(test_X, test_Y, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

Train: 0.844, Test: 0.602


In [21]:
'''
Single LSTM layer without pooling
5 epochs: sex
Train: 0.830, Test: 0.868
5 epochs: asd
Train: 0.723, Test: 0.527
10 epochs: asd
'''
'''
Single LSTM layer without pooling
5 epochs: asd
Train: 0.675, Test: 0.470
10 epochs: asd
Train: 0.799, Test: 0.579
15 epochs: asd
Train: 0.844, Test: 0.602
'''

'\nSingle LSTM layer\n5 epochs: sex\nTrain: 0.830, Test: 0.868\n5 epochs: asd\nTrain: 0.723, Test: 0.527\n10 epochs: asd\n'